In [201]:
import pandas as pd
from scipy import stats
import numpy as np
from pandas.api.types import CategoricalDtype
import re

In [202]:
pl0 = pd.read_csv("../Kraftwerksliste_2020.csv", error_bad_lines=False, engine="python", encoding="latin1", sep=";", skiprows=9)
#pl0 = pd.read_csv("Kraftwerksliste_CSV.csv", error_bad_lines=False, engine="python")
bm = pd.read_csv("block_plant_mapper.csv", error_bad_lines=False, engine="python")

In [203]:
pl0.rename(columns={ pl0.columns[10]: "Energieträger_old", pl0.columns[13]: "Energieträger"}, inplace=True)

In [59]:
#pl0

In [60]:
#bm

In [61]:
#pl0.groupby("Energieträger").count()

In [62]:
#bm
pl1 = pl0.loc[np.logical_or(pl0.Energieträger == "Kernenergie", np.logical_or(pl0.Energieträger == "Erdgas", np.logical_or(pl0.Energieträger == "Steinkohle", np.logical_or(pl0.Energieträger == "Braunkohle", pl0.Energieträger == "Mineralölprodukte")))), :]

In [63]:
plt = pl1.dropna(subset=["Kraftwerksnummer Bundesnetzagentur"])

In [64]:
plt = plt.astype({"Energieträger": 'category'})

In [65]:
#plt["Energieträger"] = plt["Energieträger"].astype('category')
#plt["Energieträger"] = plt["Energieträger"].cat.set_categories(["Kernenergie", "Braunkohle", "Steinkohle", "Erdgas", "Mineralölprodukte"], ordered=True)

In [66]:
#plt["Energieträger"] = plt["Energieträger"].astype('category')

In [67]:
#plt.groupby('Energieträger').count()

In [68]:
pl2 = plt.loc[:]

In [69]:
#pl2.groupby("Energieträger").count()

In [70]:
# plt.dtypes

In [71]:
pl2 = plt.rename(columns={ pl1.columns[0]: "blockid", pl1.columns[3]: "PLZ", pl1.columns[4]: "Ort", pl1.columns[8]: "Inbetriebnahme", pl1.columns[9]: "Status", pl1.columns[15]: "KWK", pl1.columns[16]: "Nettoleistung"})
#pl3 = pl2.rename(columns={ pl1.columns[1]: "BlockID" })

In [72]:
def to_year(x):
    if type(x) == type(4):
        return x
    elif type(x) == type(2.5):
        return 0 # handle nans
    return int(x[-4:])

In [73]:
#blocks.loc[blocks.Energieträger == "Kernenergie"]

In [74]:
pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace('.','')
pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace(',','.')
pl2.loc[pl2.blockid == 'BNA0861a', 'Inbetriebnahme'] = 2012
pl2.loc[pl2.blockid == 'BNA1334', 'Inbetriebnahme'] = 2002
pl2.loc[pl2.blockid == 'BNA1141', 'Inbetriebnahme'] = 1970
pl2.loc[pl2.blockid == 'BNA0418', 'Inbetriebnahme'] = 2013
pl2.loc[pl2.blockid == 'BNA1499', 'Inbetriebnahme'] = 1951
pl2.loc[pl2.blockid == 'BNA1502', 'Inbetriebnahme'] = 2013
pl2.loc[pl2.blockid == 'BNA1500', 'Inbetriebnahme'] = 1990
pl2.loc[pl2.blockid == 'BNA1498', 'Inbetriebnahme'] = 1953
pl2.loc[pl2.blockid == 'BNA1260', 'Inbetriebnahme'] = 2013
pl2.loc[pl2.blockid == 'BNA1056', 'Inbetriebnahme'] = 2006


#pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace('\n','')
#pl2['Nettoleistung'] = pl2[pd.to_numeric(pl2["Nettoleistung"])]
#pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace(';','.')

In [75]:
#pl2[] = pl2[pd.to_numeric(pl2["Nettoleistung"])]
pl2['Inbetriebnahme'] = pl2['Inbetriebnahme'].apply(to_year)
pl2['Nettoleistung'] = pl2['Nettoleistung'].apply(pd.to_numeric)

In [76]:
#bm[pl2.duplicated(['BlockID'], keep=False)].sort_values("BlockID", ascending=False)

In [157]:
#pl2[pl2.duplicated(['blockid'], keep=False)].sort_values("blockid", ascending=False)

In [169]:
cols = [10, 11,12,14,17,18,19]
pl3 = pl2.drop(pl2.columns[cols],axis=1)

In [177]:
#pl3.groupby('Unternehmen').max()

In [170]:
pl4 = pl3.copy()

In [221]:
def fix_company(company):
    #print(company)
    company_dict = {"RWE": "RWE AG", "Vattenfall": "Vattenfall GmbH", "Uniper": "Uniper SE", "EnBW": "EnBW AG", "Steag": "Steag GmbH", "Nordzucker": "Nordzucker AG", "Lausitz Energie": "LEAG"}
    for key, value in company_dict.items():
        if key in str(company):
            return value
    return company

In [222]:
z = "Endgültig Stillgelegt 2015"

In [223]:
def extract_still(x):
    match = re.findall("[0-9]{4}",x)
    return match[0] if match else np.nan

In [224]:
def extract_state(state):
    if state == "In Betrieb":
        return "in Betrieb"
    elif "Endgültig Stillgelegt" in state:
        return "stillgelegt"
    elif "Vorläufig Stillgelegt" in state:
        return "vorläufig stillgelegt"
    else:
        return state

In [225]:
extract_state(z)

'stillgelegt'

In [268]:
def fix_kwk(x):
    return "Nein" if x == "nein" else "Ja" if x == "ja" else x

In [269]:
#pl4 = pl3.ap

In [270]:
pl4['Unternehmen'] = pl4['Unternehmen'].apply(lambda x: fix_company(x))
pl4['Stilllegung'] = pl4['Status'].apply(lambda x: extract_still(x))
pl4['Status'] = pl4['Status'].apply(lambda x: extract_state(x))
pl4['KWK'] = pl4['KWK'].apply(lambda x: fix_kwk(x))

In [271]:
newdf = pl4.copy()

In [272]:
newdf.dtypes

blockid                                         object
Unternehmen                                     object
Kraftwerksname                                  object
PLZ                                             object
Ort                                             object
Straße und Hausnummer (Standort Kraftwerk)      object
Bundesland                                      object
Blockname                                       object
Inbetriebnahme                                   int64
Status                                          object
Energieträger                                 category
KWK                                             object
Nettoleistung                                  float64
Stilllegung                                    float64
dtype: object

In [273]:
newdf["Status"] = newdf["Status"].astype('category')
newdf["Status"] = newdf["Status"].cat.set_categories(['in Betrieb', 'Gesetzlich an Stilllegung gehindert', 'Netzreserve',  'Sicherheitsbereitschaft', 'Sonderfall', 'vorläufig stillgelegt', 'stillgelegt'], ordered=True)

In [274]:
newdf["KWK"] = newdf["KWK"].astype('category')
newdf["KWK"] = newdf["KWK"].cat.set_categories(['Ja', 'Nein'], ordered=True)

In [275]:
atest = newdf.groupby("KWK")

In [276]:
newdf.groupby("Energieträger").count()

,blockid,Unternehmen,Kraftwerksname,PLZ,Ort,Straße und Hausnummer (Standort Kraftwerk),Bundesland,Blockname,Inbetriebnahme,Status,KWK,Nettoleistung,Stilllegung
Energieträger,,,,,,,,,,,,,
Braunkohle,77,77,76,77,77,26,77,69,77,77,77,77,0
Erdgas,315,312,304,315,315,172,315,216,315,312,315,315,0
Kernenergie,9,9,9,9,9,2,9,9,9,9,9,9,0
Mineralölprodukte,56,56,56,56,56,30,56,47,56,51,55,56,0
Steinkohle,120,120,119,120,120,75,120,107,120,112,119,120,0


In [277]:
atest.count()

,blockid,Unternehmen,Kraftwerksname,PLZ,Ort,Straße und Hausnummer (Standort Kraftwerk),Bundesland,Blockname,Inbetriebnahme,Status,Energieträger,Nettoleistung,Stilllegung
KWK,,,,,,,,,,,,,
Ja,410,407,397,410,410,243,410,297,410,406,410,410,0
Nein,165,165,165,165,165,61,165,149,165,153,165,165,0


In [278]:
pl3 = newdf

In [279]:
atest = newdf.groupby(["Unternehmen"]).count()
atest.sort_values("Inbetriebnahme", inplace=True, ascending=False)

In [280]:
#print(atest)

In [281]:
atest = pl3.groupby(["Status"]).count()
# atest

In [282]:
# pl3.rename(columns={ pl3.columns[10]: "Energieträger"}, inplace=True)

In [283]:
bm[bm.duplicated(['blockid'], keep=False)].sort_values("blockid", ascending=False)

,plantid,blockid


In [284]:
#bm2 = bm.drop_duplicates()

In [285]:
pl4 = bm.merge(pl3, how="outer", on="blockid")

In [286]:
#pl4[pl4.duplicated(['BlocID'], keep=False)].sort_values("BlockID", ascending=False)

In [287]:
pl5 = pl4[pd.notnull(pl4['blockid'])]
#pl5 = pl4
pl6 = pl5[pd.notnull(pl5['Nettoleistung'])]

In [288]:
# pl6

In [290]:
pl6['plantid'] = pl6['plantid'].apply(lambda x: str(x).replace('/', '_'))

/opt/conda/envs/idp/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [291]:
# pl7 = pl6.replace("/", "_")

In [292]:
#pl7 = pl6.apply(str.replace("/", "_"))

In [293]:
#pl6.loc[pl6.Kraftwerksname == "KW Mittelsbüren"]

In [295]:
pl6['Inbetriebnahme'] = pl6['Inbetriebnahme'].apply(pd.to_numeric)

/opt/conda/envs/idp/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [296]:
blocks = pl6.copy()
stammdaten = pl6.copy()

In [297]:
blocks.sort_values("Inbetriebnahme", inplace=True, ascending=False)
# blocks

In [298]:
plants = blocks.loc[:]
plants = plants.dropna(subset=["plantid"])

In [299]:
#plants_d = plants.loc[plants.duplicated("KraftwerkID", keep=False)]
# plants_d

In [300]:
#plants_c = plants_d.groupby(["KraftwerkID"]).sum()
#plants_count = plants_d.groupby(["KraftwerkID"]).count()

In [301]:
#plants_f = plants_c.reset_index()

In [302]:
#plants_count = plants_count.loc[:, ["blockid"]]

In [303]:
#plants_count = plants_count.reset_index()
# plants_count 

In [304]:
#plants_f = plants_f.merge(plants_count, on="KraftwerkID", how="inner")
# plants_f

In [305]:
#plants[plants.duplicated(['KraftwerkID'], keep=False)].sort_values("BlockID", ascending=False)

In [306]:
plants["Energieträger"] = plants["Energieträger"].astype('category')
plants["Energieträger"] = plants["Energieträger"].cat.set_categories(["Kernenergie", "Braunkohle", "Steinkohle", "Erdgas", "Mineralölprodukte"], ordered=True)
plants["Status"] = plants["Status"].astype('category')
plants["Status"] = plants["Status"].cat.set_categories(['in Betrieb', 'Gesetzlich an Stilllegung gehindert', 'Netzreserve',  'Sicherheitsbereitschaft', 'Sonderfall', 'vorläufig stillgelegt', 'stillgelegt'], ordered=True)
plants["KWK"] = plants["KWK"].astype('category')
plants["KWK"] = plants["KWK"].cat.set_categories(['Ja', 'Nein'], ordered=True)
plants["Bundesland"] = plants["Bundesland"].astype('category')
plants["Bundesland"] = plants["Bundesland"].cat.set_categories(['Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen', 'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen', 'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 'Sachsen', 'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen'])

In [307]:
p_grp = plants.groupby("plantid")

In [308]:
plants.groupby("Energieträger").max()

,plantid,blockid,PLZ,Ort,Inbetriebnahme,Status,KWK,Nettoleistung,Stilllegung
Energieträger,,,,,,,,,
Kernenergie,666-18,BNA0802,97506,Philippsburg,1989.0,stillgelegt,Nein,1410.0,NaN
Braunkohle,664-08,BNA1511a,9116,Zülpich,2014.0,stillgelegt,Nein,1060.0,NaN
Steinkohle,663-07,BNA1674,A-44655,Zolling,2015.0,stillgelegt,Nein,875.0,NaN
Erdgas,nan,BNa1947,A-91052,Zülpich,2019.0,stillgelegt,Nein,887.0,NaN
Mineralölprodukte,669-01,BNA1338,94469,Zolling,2011.0,stillgelegt,Nein,415.0,NaN


In [309]:
plants_a = pd.DataFrame()
for plantid, group in p_grp:
    #if not plantid == "06-05-300-9046797":
    #    continue
    entry = {}
    #print(entry)
    entry["plantid"] = plantid
    # entry["BlockID"] = group
    try:
        entry["plantname"] = group["Kraftwerksname"].value_counts().index[0]
    except IndexError:
        entry["plantname"] = np.NaN
    entry["federalstate"] = group["Bundesland"].value_counts().index[0]
    entry["energysource"] = group["Energieträger"].min()
    try:
        entry["chp"] = group["KWK"].min()
    except TypeError:
        #print("aneror")
        #print(plantid)
        #print(group['KWK'].count())
        entry["chp"] = ""
    entry["latestexpanded"] = group["Inbetriebnahme"].max()
    entry["initialop"] = group["Inbetriebnahme"].min()
    entry["totalpower"] = group["Nettoleistung"].sum()
    entry["state"] = group["Status"].min()
    entry["blockcount"] = group["blockid"].count()
    try:
        entry["company"] = group["Unternehmen"].value_counts().index[0]
    except IndexError:
        ;
    #print(entry)
    #break
    plants_a = plants_a.append(entry, ignore_index=True)

In [310]:
int_list = ['blockcount', 'initialop', 'latestexpanded']
for column in int_list:
    plants_a[column] = plants_a[column].astype(int)
#plant['blockcount'] = pl2['blockcount'].apply(pd.to_numeric)
#pl2['blockcount'] = pl2['blockcount'].apply(pd.to_numeric)
#pl2['blockcount'] = pl2['blockcount'].apply(pd.to_numeric)
#pl2['blockcount'] = pl2['blockcount'].apply(pd.to_numeric)

In [311]:
plants_a.dtypes

blockcount          int64
chp                object
company            object
energysource       object
federalstate       object
initialop           int64
latestexpanded      int64
plantid            object
plantname          object
state              object
totalpower        float64
dtype: object

In [312]:
plants_a["energysource"] = plants_a["energysource"].astype('category')
plants_a["energysource"] = plants_a["energysource"].cat.set_categories(["Kernenergie", "Braunkohle", "Steinkohle", "Erdgas", "Mineralölprodukte"], ordered=True)
plants_a["state"] = plants_a["state"].astype('category')
plants_a["state"] = plants_a["state"].cat.set_categories(['in Betrieb', 'Gesetzlich an Stilllegung gehindert', 'Netzreserve',  'Sicherheitsbereitschaft', 'Sonderfall', 'vorläufig stillgelegt', 'stillgelegt'], ordered=True)
plants_a["chp"] = plants_a["chp"].astype('category')
plants_a["chp"] = plants_a["chp"].cat.set_categories(['Ja', 'Nein'], ordered=True)
plants_a["federalstate"] = plants_a["federalstate"].astype('category')
plants_a["federalstate"] = plants_a["federalstate"].cat.set_categories(['Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen', 'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen', 'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 'Sachsen', 'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen'])

In [313]:
plants_a.dtypes

blockcount           int64
chp               category
company             object
energysource      category
federalstate      category
initialop            int64
latestexpanded       int64
plantid             object
plantname           object
state             category
totalpower         float64
dtype: object

In [314]:
plants_a.loc[plants_a.plantid == "06-05-300-0326774"]

,blockcount,chp,company,energysource,federalstate,initialop,latestexpanded,plantid,plantname,state,totalpower
52,9,Ja,RWE AG,Braunkohle,Nordrhein-Westfalen,1963,2002,06-05-300-0326774,Niederaußem,in Betrieb,3656.0


In [315]:
plants_a.loc[plants_a.plantid == "06-00176010435"]

,blockcount,chp,company,energysource,federalstate,initialop,latestexpanded,plantid,plantname,state,totalpower
22,4,Ja,Uniper SE,Steinkohle,Hessen,1965,1992,06-00176010435,Staudinger,in Betrieb,1624.0


In [316]:
plants_a.loc[plants_a.plantid == "06-05-900-0865327"]

,blockcount,chp,company,energysource,federalstate,initialop,latestexpanded,plantid,plantname,state,totalpower
76,9,Nein,RWE AG,Steinkohle,Nordrhein-Westfalen,1973,1984,06-05-900-0865327,Gersteinwerk,in Betrieb,2011.0


In [317]:
#plants_a = plants.drop_duplicates(subset="KraftwerkID")

In [318]:
column_titles = ['plantid', 'plantname', 'federalstate','energysource', 'chp', 'latestexpanded', 'initialop', 'totalpower', 'state', 'blockcount', 'company']
plants_b = plants_a.reindex(columns=column_titles)

In [319]:
#plants_b

In [320]:
blocks.drop(['Ort', 'PLZ', 'Straße und Hausnummer (Standort Kraftwerk)'], axis=1, inplace=True)

In [321]:
blocks[blocks.duplicated(['blockid'], keep=False)].sort_values("blockid", ascending=False)

,plantid,blockid,Unternehmen,Kraftwerksname,Bundesland,Blockname,Inbetriebnahme,Status,Energieträger,KWK,Nettoleistung,Stilllegung
535,06-11-01-1105604,BNA0085b,Vattenfall GmbH,Moabit,Berlin,Moabit GT 7,1971.0,in Betrieb,Mineralölprodukte,Nein,34.0,NaN
536,06-11-01-1105604,BNA0085b,Vattenfall GmbH,Moabit,Berlin,Moabit GT 7,1971.0,stillgelegt,Mineralölprodukte,Nein,17.0,NaN
551,662-05,BNA0083,Vattenfall GmbH,Wilmersdorf,Berlin,Wilmersdorf GT1,1977.0,stillgelegt,Mineralölprodukte,Ja,92.0,NaN
550,662-05,BNA0083,Vattenfall GmbH,Wilmersdorf,Berlin,Wilmersdorf GT2 und GT3,1977.0,in Betrieb,Mineralölprodukte,Ja,184.0,NaN


In [322]:
#blocks

In [323]:
plants_b.sort_values("initialop", ascending=False, inplace=True)

In [324]:
#drop_list = ['Blockname', 'KWK', 'Nettoleistung', 'Stilllegung', 'Straße und Hausnummer (Standort Kraftwerk)', 'PLZ', 'Ort', 'Nettoleistung']
#plants_g = plants_f.drop(drop_list, axis=1)

In [325]:
#plants_h = plants_g.rename(columns={"KraftwerkID": "plantid", "Gesamtleistung": "totalpower", "Blockzahl": "blockcount", "BlockID": "blockid", "Bundesland": "federalstate", "Energieträger": "energysource", "Inbetriebnahme": "latestexpanded", "Kraftwerksname": "plantname", "Status": "state", "Unternehmen": "company"})

In [326]:
plants_final = plants_b.loc[:]

In [327]:
plants_final.dtypes

plantid             object
plantname           object
federalstate      category
energysource      category
chp               category
latestexpanded       int64
initialop            int64
totalpower         float64
state             category
blockcount           int64
company             object
dtype: object

In [328]:
# plants_final

In [329]:
# stammdaten

In [330]:
# cols = [0,2,3,9,10,11,12]
drop_list = ['plantid', 'Blockname', 'Energieträger', 'Inbetriebnahme', 'KWK', 'Kraftwerksname', 'Nettoleistung', 'Status', 'Stilllegung', 'Unternehmen']
stammdaten.drop(drop_list, axis=1, inplace=True)

In [331]:
#stammdaten.to_csv("stammdaten.csv", index=False)

In [332]:
#plants_final.to_csv("plants.csv", index=False)

In [333]:
#blocks.to_csv("blocks_3.csv", index=False)

In [334]:
stammdaten.to_csv("stammdaten_nh.csv", index=False, header=False)
blocks.to_csv("blocks_nh.csv", index=False, header=False)
plants_final.to_csv("plants_nh.csv", index=False, header=False)

In [335]:
#bm2.to_csv("bpm.csv", index=False)

In [336]:
#blocks

In [337]:
blocks.groupby('Energieträger').count()

,plantid,blockid,Unternehmen,Kraftwerksname,Bundesland,Blockname,Inbetriebnahme,Status,KWK,Nettoleistung,Stilllegung
Energieträger,,,,,,,,,,,
Braunkohle,77,77,77,76,77,69,77,77,77,77,0
Erdgas,315,315,312,304,315,216,315,312,315,315,0
Kernenergie,9,9,9,9,9,9,9,9,9,9,0
Mineralölprodukte,56,56,56,56,56,47,56,51,55,56,0
Steinkohle,120,120,120,119,120,107,120,112,119,120,0


In [338]:
pl4 = blocks.loc[blocks['Energieträger'] == "Mineralölprodukte"]

In [154]:
#pl4

In [155]:
#pd.set_option('display.max_rows', None)

In [339]:
pl4.loc[pl4['Energieträger'] == "Mineralölprodukte"].sort_values(["Bundesland", "Unternehmen", "Kraftwerksname"])

,plantid,blockid,Unternehmen,Kraftwerksname,Bundesland,Blockname,Inbetriebnahme,Status,Energieträger,KWK,Nettoleistung,Stilllegung
547,662-03,BNA0648,EnBW AG,Dampfkraftwerk Marbach am Neckar,Baden-Württemberg,Marbach III GT (solo),1975.0,NaN,Mineralölprodukte,Nein,85.0,NaN
548,662-03,BNA0649,EnBW AG,Dampfkraftwerk Marbach am Neckar,Baden-Württemberg,MAR III DT,1975.0,NaN,Mineralölprodukte,Nein,263.5,NaN
552,662-06,BNA0647,EnBW AG,Dampfkraftwerk Marbach am Neckar,Baden-Württemberg,Marbach II GT,1971.0,NaN,Mineralölprodukte,Nein,77.4,NaN
549,662-04,BNA1004,EnBW AG,Kraftwerk Walheim,Baden-Württemberg,WAL GT D,1981.0,in Betrieb,Mineralölprodukte,Nein,136.0,NaN
559,662-12,BNA0939b,EnBW AG,Restmüll-Heizkraftwerk Stuttgart-Münster,Baden-Württemberg,MÜN GT18,1974.0,in Betrieb,Mineralölprodukte,Nein,23.3,NaN
561,662-12,BNA0937,EnBW AG,Restmüll-Heizkraftwerk Stuttgart-Münster,Baden-Württemberg,MÜN GT16,1974.0,in Betrieb,Mineralölprodukte,Nein,23.3,NaN
560,662-12,BNA0938,EnBW AG,Restmüll-Heizkraftwerk Stuttgart-Münster,Baden-Württemberg,MÜN GT17,1974.0,in Betrieb,Mineralölprodukte,Nein,23.3,NaN
562,662-13,BNA0516,MiRO Mineraloelraffinerie Oberrhein GmbH & Co. KG,MiRO,Baden-Württemberg,Kesselhaus Werk 1,1995.0,in Betrieb,Mineralölprodukte,Ja,45.0,NaN
563,662-13,BNA0517,MiRO Mineraloelraffinerie Oberrhein GmbH & Co. KG,MiRO,Baden-Württemberg,Kesselhaus Werk 2,1995.0,in Betrieb,Mineralölprodukte,Ja,25.0,NaN
572,662-19,BNA1007b,Allgäuer Überlandwerk GmbH,SKW Diesel,Bayern,SKW Diesel,1978.0,in Betrieb,Mineralölprodukte,Nein,10.5,NaN
